# try 1

## import

In [1]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd

## 파일 로드

In [2]:
raw_tr = pd.read_json("./raw_data/train.json")
raw_te = pd.read_json("./raw_data/val.json")

In [3]:
raw_df = pd.concat([raw_tr,raw_te])

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
raw_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_tags = list(raw_df.tags)
te_tag_df = raw_tags_mlb.fit_transform(raw_tags)
tag_classes = raw_tags_mlb.classes_

raw_songs_mlb = MultiLabelBinarizer(sparse_output=True)
raw_songs = list(raw_df.songs)
te_songs_df = raw_songs_mlb.fit_transform(raw_songs)
songs_classes = raw_songs_mlb.classes_

In [5]:
tag_classes

array(['', '00', '007', ..., '힛뎀포크', '힛뎀폭', '힛뎀폭스'], dtype=object)

In [6]:
songs_classes

array([     0,      3,      4, ..., 707986, 707987, 707988])

In [7]:
te_tag_df

<138086x30197 sparse matrix of type '<class 'numpy.int64'>'
	with 503669 stored elements in Compressed Sparse Row format>

In [8]:
te_songs_df

<138086x638336 sparse matrix of type '<class 'numpy.int64'>'
	with 5707070 stored elements in Compressed Sparse Row format>

# 모델링
- 1. sparse 행렬 작업
- 2. 두 행렬을 합치고
- 3. 모델 fit 시키기

In [9]:
import scipy.sparse
r = scipy.sparse.hstack([te_tag_df, te_songs_df])
r = csr_matrix(r)

In [10]:
r

<138086x668533 sparse matrix of type '<class 'numpy.int64'>'
	with 6210739 stored elements in Compressed Sparse Row format>

In [11]:
from scipy.sparse import csr_matrix
like_cnt = csr_matrix(raw_df.like_cnt)
like_cnt

<1x138086 sparse matrix of type '<class 'numpy.int64'>'
	with 125718 stored elements in Compressed Sparse Row format>

In [12]:
like_cnt.T

<138086x1 sparse matrix of type '<class 'numpy.int64'>'
	with 125718 stored elements in Compressed Sparse Column format>

In [13]:
r

<138086x668533 sparse matrix of type '<class 'numpy.int64'>'
	with 6210739 stored elements in Compressed Sparse Row format>

In [14]:
r = scipy.sparse.hstack([r,like_cnt.T])
r

<138086x668534 sparse matrix of type '<class 'numpy.int64'>'
	with 6336457 stored elements in COOrdinate format>

# LogisticMatrixFactorization 시도

In [15]:
import implicit

In [21]:
from implicit.lmf import LogisticMatrixFactorization as LMF
LMF_model = LMF(factors=256, regularization=0.08, iterations=10)
LMF_model.fit(r.T * 15)

KeyboardInterrupt: 

In [ ]:
als_model = ALS(factors=256, regularization=0.08, iterations=10, calculate_training_loss=True)
als_model.fit(r.T * 15.0)

### 노래 예측 모댈과, 태그 예측모댈에 user factor와 item factor를 분배

In [ ]:
n_items = 638336

In [ ]:
item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [ ]:
item_model.item_factors = als_model.item_factors[:n_items]
tag_model.item_factors = als_model.item_factors[n_items:-1]

In [ ]:
als_model.item_factors.shape

In [ ]:
als_model.item_factors[n_items:]

In [ ]:
raw_tr

In [31]:
tr_csr = r.todense()[:115071]

In [ ]:
tr_csr = sp.sparse.csr_matrix(tr_csr)

In [ ]:
tr_csr.shape

In [ ]:
tr_csr
# test부분의 데이터 자르고 위엣 부분만 따로 만들어야 함

In [54]:
item_rec_csr = tr_csr[:, :n_items]
tag_rec_csr = tr_csr[:, n_items:]

## recommend 과정

In [ ]:
idx_to_tag
# dict로 만들면 됨

In [ ]:
idx_to_iid

In [55]:
item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(te_csr.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=100)
    item_rec = [idx_to_iid[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=100)
    tag_rec = [idx_to_tag[x[0]] for x in tag_rec if x[0] in idx_to_tag]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [66]:
tag_rec_csr

<115071x29160 sparse matrix of type '<class 'numpy.int64'>'
	with 476331 stored elements in Compressed Sparse Row format>

In [65]:
item_rec_csr

<115071x615142 sparse matrix of type '<class 'numpy.int64'>'
	with 5285871 stored elements in Compressed Sparse Row format>

In [35]:
tag_model.item_factors

array([[ 1.9774567e-01, -1.6457781e-01, -2.4069002e-02, ...,
         3.8423190e-01,  2.2601548e-01,  5.1538068e-01],
       [-2.3180166e-01, -4.1238788e-01,  1.8979147e-01, ...,
         2.5333425e-01,  1.1671105e-01,  6.6225190e-04],
       [-1.3898441e-01, -3.7492254e-01,  1.6532259e-01, ...,
         2.5847656e-01,  8.5519031e-02,  1.0789641e-02],
       ...,
       [ 6.0895982e-04,  1.2598315e-03,  1.5657897e-04, ...,
         6.7041896e-04,  8.2416815e-04,  2.6454786e-03],
       [ 6.7919539e-04, -7.3085364e-04,  1.4212606e-03, ...,
        -7.8746898e-04, -7.3015870e-04,  1.2854089e-03],
       [ 9.5603382e-04,  6.5275340e-04,  4.8686855e-04, ...,
        -7.2426465e-04, -1.5322662e-04,  3.2020046e-04]], dtype=float32)

In [26]:
tag_model.item_factors

array([[ 3.55746090e-01, -5.69548428e-01, -5.59754260e-02, ...,
         4.83457536e-01,  1.95368275e-01,  1.14752978e-01],
       [ 1.13077201e-01,  4.99642581e-01,  1.01401411e-01, ...,
         8.70525062e-01,  5.08861303e-01,  4.67959046e-01],
       [ 1.34080434e-02,  5.13347864e-01,  1.56607270e-01, ...,
         6.01837337e-01,  4.08071131e-01,  3.32719356e-01],
       ...,
       [-5.10524551e-04,  3.28546972e-04,  8.81741638e-04, ...,
         8.72330857e-04,  1.68862971e-04,  1.18788111e-03],
       [ 9.88858286e-04,  1.02399255e-03,  3.29685328e-03, ...,
         1.42479164e-03, -1.69632386e-03, -1.65981124e-03],
       [ 9.73270435e-05,  1.62653669e-04, -2.30579590e-05, ...,
         1.98975016e-04,  2.20649337e-04, -1.06205756e-04]], dtype=float32)

## 최종 json파일 추출

In [56]:
returnval = []
for _id, rec, tag_rec in zip(test_ids, item_ret, tag_ret):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [57]:
import json
with open('results.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))